In [1]:
from crewai import Agent, Crew, Process, Task
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from datetime import datetime, timedelta
from typing import Union, List, Tuple, Dict
from langchain_core.agents import AgentFinish

In [2]:
import os
from utils import get_groq_api_key, get_openai_model, get_openai_api_key
os.environ["OPENAI_MODEL_NAME"] = get_openai_model()
os.environ["OPENAI_API_KEY"] = get_openai_api_key()
os.environ["GROQ_API_KEY"] = get_groq_api_key()

In [15]:
def get_llm(name, temp):
    if name == "openai":
        return ChatOpenAI(temperature=temp, model=get_openai_model())
    elif name == "llama":
        return ChatGroq(model="llama3-70b-8192")

In [4]:
from crewai_tools import BaseTool
from exa_py import Exa
from utils import get_exa_api_key

class SearchContents(BaseTool):
    name : str = "Search and Contents Tool"
    description : str = ("Perform an search given an input query and retrieve a list of relevant results as links, text and highlights of the content")

    def _run(self, search_query: str) -> str:
        if isinstance(search_query, dict):
            search_query = search_query['title']
        exa = Exa(api_key=get_exa_api_key())
        one_week = datetime.now() - timedelta(days=7)
        date_cutoff = one_week.strftime("%Y-%m-%d")

        results = exa.search_and_contents(
            query=search_query,
            highlights=True,
            use_autoprompt=True,
            start_published_date=date_cutoff,
            text={"include_html_tags": False, "max_characters": 8000},
            num_results=7
        )
        return results


class SimilarResultsContents(BaseTool):
    name : str = "Find similar Tool"
    description : str = ("Find a list of similar results based on a webpage's URL, including the text content or highlights of each result")

    def _run(self, web_url: str) -> str:
        exa = Exa(api_key=get_exa_api_key())
        one_week = datetime.now() - timedelta(days=7)
        date_cutoff = one_week.strftime("%Y-%m-%d")

        results = exa.find_similar_and_contents(
            url=web_url,
            highlights=True,
            start_published_date=date_cutoff 
        )
        return results


In [5]:
search_content = SearchContents()
similar_results = SimilarResultsContents()

In [16]:
researcher = Agent(
    role="Senior Researcher",
    goal="Uncover cutting-edge developments on {topic}",
    llm=get_llm("openai", 0),
    backstory=("You're a seasoned journalist with a nose for news, who scores the digital landscape for leads"
               "You're known for your great research skills and ability to dig up the most interesting stories." 
               "Your reports are always thorough and well-researched, making you a trusted source of information."
               "You always follow the rules and guidelines provided to you and you never forget to include the complete URL, title, and insightful summary of the article where you found the news."
               "Your collect information that helps the audience learn something."
               "Your work is the basis for the Content Writer to write a newsletter on this topic"
               ),
    tools=[search_content],
    allow_delegation=False,
    verbose=True,
)

editor = Agent(
    role="Editor",
    goal="Ensure the quality of the final newsletter",
    llm=get_llm("openai", 0.5),
    backstory=("You are the Editor of a prestigious news organization."
               "You are responsible for overseeing the production of the newsletter and ensuring that it is well-written, and engaging."
               "You review the articles provided by the Senior Researcher, ensuring the content is engaging and relevant."
               "You add context to each article, explaining its relevance and importance to the readers."
               "With a keen sense of what resonates with your audience, you reorder the articles to place the most important news at the top."
    ),
    #tools=[search_content, similar_results],
    allow_delegation=False,
    verbose=True,
)

writer = Agent(
    role="Content Writer",
    goal="Transform the research findings into engaging and informative articles",
    llm=get_llm("openai", 0.7),
    backstory=("You are a talented writer with a knack for storytelling."
               "Your job is to take the thorough research provided by the Senior Researcher and Editor and turn it into compelling articles that capture the reader's attention."
               "You ensure that the content is clear, well-structured, and free of any grammatical errors."
    ),
    allow_delegation=False,
    verbose=True,
)

developer = Agent(
    role="HTML Newsletter Developer",
    goal="Create the final newsletter document in correct HTML format with all news articles, urls and links properly integrated",
    llm=get_llm("openai", 0),
    backstory=("You are responsible for developing the newsletter from scratch using HTML."
               "You take the articles and content provided by the Content Writer and format them into a cohesive and visually appealing newsletter."
               "Your job involves writing HTML code to ensure that each article is properly integrated, maintaining a consistent design and layout."
               "You ensure all links and urls are correctly embedded and functional, providing a seamless reading experience for the audience."
               "Your role is crucial in transforming the newsletter content into a polished and professional final product."
    ),
    allow_delegation=False,
    verbose=True,
)

In [17]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a newsletter "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True,
    llm=get_llm("openai", 0),
    tools=[search_content]
)

writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm=get_llm("openai", 0.7),
)

editor = Agent(
    role="Editor",
    goal="Edit a given newsletter to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the newsletter "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm=get_llm("openai", 0.5),
)

In [18]:
research_task = Task(
    description=("Conduct a thorough research about the latest news on {topic}. Be sure to look for sources that are reliable and publish recent news. Do not include articles that are not news material or that are not directly related to {topic}."
                 "With this research, compile a list of the most relevant news stories that you found."
                 "Follow these rules: \n"
                 "- Summarize the news in a few sentences. Make the summary as long as necessary to include all the relevant information, but not too long for a newsletter.\n"
                 "- Include the URL of the article where you found the news.\n"
                 "- Include a minimum of 7 news articles and a maximum of 10 news articles.\n"
                 "- When using the search_content Tool, your search query should be concise\n"
                 "- When using the search_content Tool only pass in the query and not a dictionary\n"
                   
    ),
    expected_output=(
        "A markdown document containing the most relevant news stories. Each news story should contain the following: \n"
        "- Title of the news\n"
        "- URL of the article where the news was found\n"
        "- Highlights of the news\n"
    ),
    agent=planner,
    output_file=f"./logs/{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}_research_task.md"
)

In [19]:
editor_task = Task(
    description=("Proofread the given newsletter for "
                 "grammatical errors and "
                 "alignment with the brand's voice."
                 "Give the newsletter an overall title related to the curated news of the week."
                 "Do not remove any url or links."
                 "Do not search for additional news articles or change the content of the news articles. Only edit the existing news articles."
    ),
    expected_output=("A well-written markdown document with all the news to be included in the newsletter of the week"
                     "Each one should contain:\n"
                     "- Title: \n"
                     "- Summary of highlights: \n"
                     "- Important 2-3 sentences about the text: \n"
                     "- Why this is important: \n"
                     "- Source: [URL of the article]"
    ),
    agent=editor,
    output_file=f"./logs/{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}_editor_task.md"
)

In [20]:
writer_task = Task(
    description=(
        "1. Use the editor document to craft a compelling "
            "newsletter document.\n"
        "2. Incorporate SEO naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the newsletter is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written newsletter "
        "in markdown format, ready for publication.",
    agent=writer,
    output_file=f"./logs/{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}_writer_task.md"
)

In [21]:
developer_task = Task(
    description=(
        "Create a HTML document with exactly the same information that is given to you. Also, include the following message at the beginning of the newsletter: {message}\n"
        "Be sure to follow these rules: \n"
        "- Do not truncate the information or change the order of the elements.\n"
        "- Do not modify the content of the news articles.\n"
        "- Do not add any additional information to the newsletter.\n"
    ),
    expected_output=("Return only the contents of the HTML template, without any tripple quotes like '```', '```html' or any other text." 
    "This output will be saved as a HTML file."),
    agent=developer,
    output_file=f"logs/{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}_newsletter_task.html"
)

In [22]:
crew = Crew(
    agents=[planner, editor, writer, developer],
    tasks=[research_task, editor_task, developer_task],
    verbose=2
)

2024-07-26 09:24:49,018 - 8123243584 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [25]:
from utils import load_html
inputs = {
        'topic': 'latest news on AI this week',
        'message': '#1',
        'html_template': load_html()
    }

In [26]:
result = crew.kickoff(inputs=inputs)

 [2024-07-26 09:31:51][DEBUG]: == Working Agent: Content Planner
 [2024-07-26 09:31:51][INFO]: == Starting Task: Conduct a thorough research about the latest news on latest news on AI this week. Be sure to look for sources that are reliable and publish recent news. Do not include articles that are not news material or that are not directly related to latest news on AI this week.With this research, compile a list of the most relevant news stories that you found.Follow these rules: 
- Summarize the news in a few sentences. Make the summary as long as necessary to include all the relevant information, but not too long for a newsletter.
- Include the URL of the article where you found the news.
- Include a minimum of 7 news articles and a maximum of 10 news articles.
- When using the search_content Tool, your search query should be concise
- When using the search_content Tool only pass in the query and not a dictionary



> Entering new CrewAgentExecutor chain...
Action: Search and Content